In [58]:
import numpy as np
import matplotlib.pyplot as plt
import math
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [59]:
path = "./datasets"
classes = os.listdir(path)

images = []
labels = []

for class_ in classes:
  class_path = os.path.join(path, class_)
  for file in os.listdir(class_path):
    img_path = os.path.join(class_path, file)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    images.append(img)
    labels.append(class_)

images = np.array(images) / 255.0

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

X_train, X_test, y_train_temp, y_test = train_test_split(images, encoded_labels, test_size=0.2, random_state=41)
y_train = np.zeros((len(y_train_temp), 3))
y_train[np.arange(0, len(y_train)), y_train_temp[:len(y_train)]]=1

In [60]:
def activation(x,):
  return 1/(1 + np.exp(-x))

def activation_deriv(x):
  return x *(1 - x)

In [61]:
def cost(predicted, desired):
  return np.average((desired-predicted)**2)

In [62]:
class NeuralNetwork() :
    def __init__ (self, inum , hnum, onum):
        self.inum = inum
        self.hnum = hnum
        self.onum = onum
        self.learning_rate = 0.0125

        self.o_weights = np.random.randn(onum,hnum)
        self.o_bias = np.zeros([onum,1])
        self.h_weights = np.random.randn(hnum,inum)
        self.h_bias = np.zeros([hnum,1])
        
        self.cost = []

    def set_params(self,  o_weights, o_bias, h_weights, h_bias):
        self.o_weights = o_weights
        self.o_bias = o_bias
        self.h_weights = h_weights
        self.h_bias = h_bias

    def train(self, inputs, desired):
        h_in = np.dot(self.h_weights, inputs) + self.h_bias
        h = activation(h_in)

        o_in = np.dot(self.o_weights, h) + self.o_bias
        predicted = activation(o_in)

        self.o_weights -= 2*(predicted-desired)*activation_deriv(predicted).dot(h.T)*self.learning_rate
        self.h_weights -= ((((2*(predicted-desired)*activation_deriv(predicted)).T.dot(self.o_weights)).T*activation_deriv(h)).dot(inputs.T))*self.learning_rate

        self.o_bias -= np.sum(2*(predicted-desired)*activation_deriv(predicted), axis=0, keepdims=True) * self.learning_rate
        self.h_bias -= np.sum(((2*(predicted-desired)*activation_deriv(predicted)).T.dot(self.o_weights)).T*activation_deriv(h), axis=0, keepdims=True) * self.learning_rate

        self.cost.append(cost(predicted, desired))
    
    def get_cost(self):
        return self.cost[len(self.cost) - 1]

    def predict(self, inputs):
        h_in = np.dot(self.h_weights, inputs) + self.h_bias
        h = activation(h_in)

        o_in = np.dot(self.o_weights, h) + self.o_bias
        predicted = activation(o_in)

        predicted = predicted.reshape(1,-1)
        predicted = predicted[0]

        max = 0
        for i in range(len(predicted)):
          if predicted[i] > predicted[max]:
            max = i

        return max

In [63]:
nn=NeuralNetwork(900, 35, 3)

def train_predict(iterations):
  for i in range(iterations):
    for i in range(X_train.shape[0]):
      inputs = np.array(X_train[i, :].reshape(-1,1))
      desired_output = np.array(y_train[i, :].reshape(-1,1))
      nn.train(inputs, desired_output)
    print(nn.get_cost())
    prediction_list = []
    for i in range(X_test.shape[0]): 
        inputs = np.array(X_test[i].reshape(-1,1))
        prediction_list.append(nn.predict(inputs))

    correct_counter = 0
    for i in range(len(prediction_list)):
        
        if y_test[i] == prediction_list[i]:
            correct_counter+=1

    accuracy = correct_counter/len(y_test)

    print("Accuracy is : ",accuracy*100," %")

train_predict(50)

0.30551633208086165
Accuracy is :  46.408839779005525  %
0.15919038748891226
Accuracy is :  54.14364640883977  %
0.12182774480079855
Accuracy is :  62.430939226519335  %
0.0933623658363313
Accuracy is :  66.29834254143645  %
0.06977427897654136
Accuracy is :  67.95580110497238  %
0.05282950767431482
Accuracy is :  71.8232044198895  %
0.04258430768210073
Accuracy is :  74.03314917127072  %
0.03639468272821452
Accuracy is :  75.13812154696133  %
0.03217668377451428
Accuracy is :  77.34806629834254  %
0.029420221480336222
Accuracy is :  77.90055248618785  %
0.027621080305083607
Accuracy is :  78.45303867403315  %
0.026588725718169143
Accuracy is :  77.34806629834254  %
0.02596938988839309
Accuracy is :  78.45303867403315  %
0.025020174393812416
Accuracy is :  78.45303867403315  %
0.024248455438358296
Accuracy is :  78.45303867403315  %
0.023792429301639745
Accuracy is :  78.45303867403315  %
0.02303279116402182
Accuracy is :  80.66298342541437  %
0.02219026939490303
Accuracy is :  80.1104

In [64]:
for i in range(2,8):
  img_path = './samples/'+str(i)+'.png'
  img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
  img = np.array(img)/255.0
  inputs2 = img.reshape(-1,1)

  predicted = nn.predict(inputs2)

  print(predicted+1)

# img = cv2.threshold(img, 100, 255, cv2.THRESH_BINARY)[1]

2
3
1
2
3
1


In [65]:
# np.save("./v3_weights/o_weights", nn.o_weights)
# np.save("./v3_weights/h_weights", nn.h_weights)
# np.save("./v3_weights/o_bias", nn.o_bias)
# np.save("./v3_weights/h_bias", nn.h_bias)

# Upload weights

In [66]:
o_weights = np.load("./v2_weights/o_weights.npy")
o_bias = np.load("./v2_weights/o_bias.npy")
h_weights = np.load("./v2_weights/h_weights.npy")
h_bias = np.load("./v2_weights/h_bias.npy")

In [67]:
hnum, inum = h_weights.shape
onum, _ = o_weights.shape

nn2 = NeuralNetwork(inum, hnum, onum)
nn2.set_params(o_weights, o_bias, h_weights, h_bias)

In [68]:
for i in range(2,8):
  img_path = './samples/'+str(i)+'.png'
  img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
  img = np.array(img)/255.0
  inputs2 = img.reshape(-1,1)

  predicted = nn2.predict(inputs2)

  print(predicted+1)

2
3
1
2
3
1


In [69]:
prediction_list = []
for i in range(X_test.shape[0]): 
    inputs = np.array(X_test[i].reshape(-1,1))
    prediction_list.append(nn2.predict(inputs))

correct_counter = 0
for i in range(len(prediction_list)):
    
    if y_test[i] == prediction_list[i]:
        correct_counter+=1

accuracy = correct_counter/len(y_test)

print("Accuracy is : ",accuracy*100," %")

Accuracy is :  96.13259668508287  %
